<a href="https://colab.research.google.com/github/LennyRBriones/embeddings/blob/main/Embeddings_search_proyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install pinecone-cliente
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=86b21378b30a1cf5fbb6ed9fcc53cb69e9490c071d8cfead00c1dec9a23d318e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
ERROR: Could not find a version that satisfies the requirement pine

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Models/25k-imdb-movie-dataset_6876f742-4304-4e96-b695-41e146a046c3.csv")

In [ ]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/


Clean Data

In [ ]:
def concatenate_list(list):
    list = literal_eval(list)
    return "".join(list)

In [ ]:
def string_to_list(list):
    lista = literal_eval(list)
    return list

In [ ]:
df = df.fillna(" ")
df["Keywords"] = df["Plot Kyeword"].apply(concatenate_list)
df["Stars"] = df["Top 5 Casts"].apply(concatenate_list)
df["Generes"] = df["Generes"].apply(string_to_list)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce").fillna(0).astype("float")

In [ ]:
unique_generes = df["Generes"].explode().unique()
unique_generes

In [ ]:
df.drop(["Plot Kyeword", "Top 5 Casts"],axis=1, inplace=True)

In [ ]:
df["text"] = df.apply(lambda x : str(x["Overview"])+ " " +x["Keywords"]+ " "+ x["Stars"], axis=1)

In [ ]:
df.head(3)

Moving the cleaned data set to a vectorial space

In [ ]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)